<a href="https://colab.research.google.com/github/krish-dd/F.M.M.L/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [16]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [17]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [18]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [19]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [20]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [21]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [22]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [23]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [24]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [25]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [26]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [27]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [28]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [29]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [30]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?




1A)Yes, averaging the validation accuracy across multiple splits generally provides more consistent results. Here’s why:

1. **Reduction in Variability**: When you average the validation accuracy over multiple data splits (e.g., in k-fold cross-validation), you reduce the impact of any single, potentially unrepresentative data split. This averaging process smooths out fluctuations caused by the random variations in different splits.

2. **Comprehensive Evaluation**: By evaluating the model on various subsets of the data, you get a more comprehensive assessment of its performance. This approach helps ensure that the model’s performance is not overly dependent on the specific characteristics of a single split.

3. **Mitigation of Overfitting**: Averaging over multiple splits helps to mitigate the risk of overfitting to a particular validation subset, leading to a more robust estimate of the model’s generalization capability.

Overall, averaging validation accuracies across multiple splits provides a more reliable and stable measure of model performance compared to using a single split.

2A)Yes, averaging validation accuracy across multiple splits, as done in techniques like *k-fold cross-validation*, typically gives a more accurate estimate of the test accuracy compared to a single train/validation split.

Here’s why:

1. **Reduced Variance:** In a single train/validation split, the performance can vary significantly based on how the data is divided. By using multiple splits, you ensure that the model is tested on different portions of the data, reducing the impact of a "lucky" or "unlucky" split.

2. **Better Generalization:** Since the model is trained and validated on different parts of the data, the average performance across these splits tends to better reflect how the model will generalize to new, unseen data (like the test set). In essence, you’re sampling different validation sets to reduce bias in your estimate.

3. **Less Overfitting to Specific Splits:** With only one validation split, the model's performance might be skewed if the split contains easier or harder examples than the test set. Multiple splits average out these effects, making the estimate of test accuracy more robust and less prone to overfitting to specific data points.

4. **More Representative of the Whole Dataset:** Cross-validation makes better use of the entire dataset, ensuring that every data point is used for both training and validation. This provides a more holistic assessment of the model’s performance across the entire data distribution, leading to a more accurate estimate of how the model will perform on the test set.

However, it's important to note that while cross-validation reduces variance and gives a more accurate estimate of test accuracy, it doesn't guarantee perfect accuracy, especially if the test data distribution differs significantly from the training data.

3A)The number of iterations (or the number of splits) in cross-validation or other similar techniques has a notable effect on the accuracy and stability of the model’s performance estimate. Here's how the number of iterations impacts the estimate:

### 1. **Increased Stability and Lower Variance**
   - **Few Iterations:** With a small number of iterations (e.g., a small number of folds in k-fold cross-validation), the performance estimate can be quite unstable. This is because fewer splits may not represent the entire data distribution well, and the model’s performance can vary significantly across different splits.
   - **More Iterations:** As you increase the number of iterations, the performance estimate tends to become more stable. Each iteration introduces more diversity in the train/validation splits, and averaging across many iterations reduces the impact of any single split. This decreases variance in the estimate.

### 2. **More Accurate Estimate of Generalization**
   - **Few Iterations:** When the number of iterations is low, the estimate of test accuracy might be biased, because certain data points may not be represented well, or some splits might inadvertently favor the model (or penalize it).
   - **More Iterations:** Increasing the number of iterations typically leads to a more accurate estimate of how the model will perform on unseen data. This is because it reduces the chance of overfitting to any particular fold or split and provides a more comprehensive evaluation across the dataset.

### 3. **Effect of Diminishing Returns**
   - Initially, increasing the number of iterations significantly improves the estimate by making it more reliable and robust. However, **diminishing returns** set in after a certain point. As you continue increasing the number of iterations (e.g., very high k in k-fold cross-validation or repeating train/test splits many times), the improvement in the accuracy of the estimate becomes minimal.
   - The added computational cost may outweigh the marginal gains in the accuracy of the estimate. In practice, increasing iterations beyond a certain point does not make a substantial difference in the estimate but increases runtime.

### 4. **Handling Small Datasets**
   - With a small dataset, a higher number of iterations can help make better use of the limited data. More iterations mean that the model is trained and validated on more data combinations, which can improve the reliability of the estimate.
   - However, with extremely small datasets, there’s a limit to how much increasing the number of iterations can help. Eventually, the model will still struggle with generalization due to the lack of diverse data.

### 5. **Computational Cost**
   - More iterations require more training cycles, leading to higher computational costs. For instance, increasing k in k-fold cross-validation means retraining the model k times. This could be expensive if the model is complex or the dataset is large.
   - You need to balance the benefit of more iterations against the increased computational cost.

### Summary
- **More iterations** generally lead to a **more stable** and **accurate estimate** of test accuracy by reducing the variance in the estimate.
- However, **diminishing returns** occur at a certain point, beyond which additional iterations offer minimal improvement in accuracy.
- Higher iterations help with small datasets but won’t fully overcome the limitations of limited data, and there’s a computational cost to consider.

A typical value for k in k-fold cross-validation is 5 or 10, which offers a good trade-off between stability and computational cost.

4A)Yes, we generally get a **better estimate** with higher iterations, but there are some nuances to consider:

### 1. **Improved Stability and Accuracy**
   - **Higher iterations** (such as increasing the number of folds in k-fold cross-validation) provide a more reliable and consistent estimate. This is because each split gives the model a chance to train and validate on different parts of the data, leading to less bias from any one split.
   - Averaging over more iterations reduces the influence of random variation in any single split, providing a more accurate reflection of the model’s true performance on unseen data.

### 2. **Reduced Variance**
   - With **fewer iterations**, the variance in performance estimates can be high, meaning the results from different validation sets can differ more dramatically.
   - As the number of iterations increases, the variance of the accuracy estimate decreases. This helps the model’s performance on the validation set better approximate its performance on an unseen test set.

### 3. **Diminishing Returns**
   - While more iterations lead to better estimates initially, there are **diminishing returns**. After a certain point, increasing the number of iterations yields only small improvements in the accuracy of the estimate.
   - For example, going from 5-fold to 10-fold cross-validation may provide noticeable improvement in the accuracy estimate. However, going from 10 to 20 folds may not provide much additional benefit, despite requiring more computational resources.

### 4. **Computational Cost**
   - **Higher iterations** also increase computational cost because the model has to be trained more times. This is particularly important for large datasets or complex models, where each additional iteration might significantly increase the training time.
   - In practice, this means you need to balance the benefit of a better estimate with the added computational expense.

### 5. **Small Datasets**
   - For **small datasets**, higher iterations (e.g., in k-fold cross-validation) can provide a better estimate because more of the data is used for training and validation. However, even with many iterations, the model’s performance might still be limited by the amount of data available.
   - Increasing iterations can mitigate, but not entirely solve, the problem of small datasets.

### Summary
- **Yes, higher iterations lead to a better estimate** of performance because they reduce variance and provide a more comprehensive evaluation of the model across different data splits.
- However, after a certain point, the improvements become marginal (**diminishing returns**), and the added computational cost may not be worth it.

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.